#  任务3:模型测试及应用部署
## 职业能力目标：

- 学习使用模型预测；

- 搭建web应用

## 任务描述

- 介绍如何加载模型

- 介绍使用模型预测物体

- 部署flask应用

## 知识储备

## 1 模型部署
模型部署一般就是把训练的模型持久化，然后运行服务器加载模型，并提供REST或其它形式的服务接口。

## 2 flask介绍

### 2.1 flask是什么
-  Flask是一个轻量级的可定制框架，使用Python语言编写，较其他同类型框架更为灵活、轻便、安全且容易上手。
-  它可以很好地结合MVC模式进行开发，开发人员分工合作，小型团队在短时间内就可以完成功能丰富的中小型网站或Web服务的实现。
-  另外，Flask还有很强的定制性，用户可以根据自己的需求来添加相应的功能，在保持核心功能简单的同时实现功能的丰富与扩展，其强大的插件库可以让用户实现个性化的网站定制，开发出功能强大的网站。

### 2.2一个完整的HTTP请求过程
域名解析 —> 与服务器建立连接 —> 发起HTTP请求 —> 服务器响应HTTP请求，浏览器得到html代码 —> 浏览器解析html代码，并请求html代码中的资源（如js、css、图片） —> 浏览器对页面进行渲染呈现给用户

## 任务实施

## 1. 部署yolo检测应用

**将任务路径更改到`back`目录下**

In [ ]:
%cd '/home/jovyan/yolo-flask-vue/back'
!ls

### 1.1 导入模块，读取配置

In [ ]:
!sudo pip install --upgrade pip

!sudo pip install -r requirements.txt -i https://pypi.douban.com/simple

### 1.2 flask安装与使用

In [ ]:
!sudo pip install flask

## 2 部署模型

### 2.1 模型部署介绍

部署yolo目标检测模型需要启动两个服务，需要启动两个终端，分别运行：

1. 后端模型服务
2. 前端页面服务


### 2.2 启动后端模型服务
**打开一个新的terminal，进入到back目录下执行以下命令部署前端**
`python3 app.py`

如下图所示：

<img style="float: center;" width = "1024" height = "600" src="./tmp/image/后端服务启动.png">

### 2.3 启动前端页面服务

**在front目录下执行以下命令部署前端**

`npm install`

`npm run serve`

如下图所示：

<img style="float: center;" width = "1024" height = "600" src="./tmp/image/前端页面服务启动.png">

### 2.4 启动云桌面，打开浏览器输入`http://localhost:8080`

**将任意一张图片放到`Desktop`文件夹下**

<img style="float: center;" width = "600" height = "200" src="./tmp/image/云桌面放入图片.png">

**点击`+`号，打开云桌面**

<img style="float: center;" width = "600" height = "200" src="./tmp/image/打开云桌面.png">

<img style="float: center;" width = "600" height = "200" src="./tmp/image/云桌面.png">

**打开浏览器如下图所示：**

<img style="float: center;" width = "800" height = "600" src="./tmp/image/桌面浏览器.png">

**最终效果如下图所示：**

<img style="float: center;" width = "800" height = "600" src="./tmp/image/检测效果.jpg">

## 3 使用flask部署模型详解

**停止后端模型服务**

本小节对后端服务的模型部署进行详细讲解，需要先关闭第一个terminal的后端模型服务

###  3.1 一个最小的 Flask 应用如下：

```python
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"
```

那么，这些代码是什么意思呢？

首先我们导入了 Flask 类。该类的实例将会成为我们的 WSGI 应用。

接着我们创建一个该类的实例。第一个参数是应用模块或者包的名称。 __name__ 是一个适用于大多数情况的快捷方式。有了这个参数， Flask 才能知道在哪里可以找到模板和静态文件等东西。

然后我们使用 route() 装饰器来告诉 Flask 触发函数 的 URL 。

函数返回需要在用户浏览器中显示的信息。默认的内容类型是 HTML ，因此字 符串中的 HTML 会被浏览器渲染。

### 3.2 搭建flask框架

**进入后端服务所在文件夹**

In [ ]:
%cd '/home/jovyan/yolo-flask-vue/back'
!ls

**导入必要的包和模块**

In [ ]:
import datetime ## 时间模块
import logging as rel_log  ## 日志模块
import os  ## 系统模块
import shutil  ##复制文件模块
from datetime import timedelta  ## 时间模块中的timedelta
from flask import *  ## flask模块

import core.main ##核心代码中的main预测函数
from processor.detector import Detector  ##加载模型类
from flask_ngrok import run_with_ngrok  ##反向代理flask模块

**设置参数**

In [ ]:
UPLOAD_FOLDER = r'./uploads'   ##上传目录

ALLOWED_EXTENSIONS = set(['png', 'jpg','jpeg'])  ##检测图片格式
app = Flask(__name__)  ##初始化flask app
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER  ##把上传目录设置在flask通用参数 
werkzeug_logger = rel_log.getLogger('werkzeug')  ## flask 日志设置
werkzeug_logger.setLevel(rel_log.ERROR)  ## 日志等级设置

app.config['SEND_FILE_MAX_AGE_DEFAULT'] = timedelta(seconds=1)  ## 解决缓存刷新问题

- `@app.after_request`：函数被响应对象调用，并且必须返回响应对象。这允许函数修改或在发送之前替换响应。

In [ ]:
# 添加header解决跨域
@app.after_request
def after_request(response):
    response.headers['Access-Control-Allow-Origin'] = '*'
    response.headers['Access-Control-Allow-Credentials'] = 'true'
    response.headers['Access-Control-Allow-Methods'] = 'POST'
    response.headers['Access-Control-Allow-Headers'] = 'Content-Type, X-Requested-With'
    return response

**设置允许访问的图片格式参数**

<font color=red size=3>动手练习1</font>

使用上述已申明变量`ALLOWED_EXTENSIONS`，请在`<1>`中写出如何判断文件是否属于允许请求的格式。

语句为：`filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS`

In [ ]:
def allowed_file(filename):
    return '.' in filename and <1>

<details>
<summary><font color=red size=3>点击查看动手练习1答案</font></summary>
<pre><code>

```python
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS
```
</code></pre>
</details>

#### 接口设置url
接口访问请求的地址
- `@app.route('/')`：装饰一个视图函数，用给定的URL注册它规则和选项。调用:meth: ' add_url_rule '，它有更多实现的细节
  参数说明：
  - `methods`：list，当前url地址,允许访问的请求方式 类型为可迭代对象,允许八种http请求方式。
  - `endpoint`：str，路由Mapping地址对应视图函数,相当于给函数起个别名。
  - `strict_slashes`:bool，url地址结尾符“ / ”的控制False: 无论结尾 “ / ” 是否存在均可以访问, True: 结尾不允许是“ / ”。
  - `defaults`: object，视图函数的参数默认值。
  - `strict_slashes`:str，url地址重定向, 浏览器的请求返回为308。
  - `subdomain`: str，子域名前缀, subdoadmin=“car”, 这样可以得到car.xxx.com 不过还需要配置。

In [ ]:
@app.route('/')
def hello_world():
    return redirect(url_for('static', filename='./index.html'))

**设置展示文件的接口**

In [ ]:
@app.route('/tmp/<path:file>', methods=['GET'])
def show_photo(file):
    if request.method == 'GET':
        if not file is None:
            image_data = open(f'tmp/{file}', "rb").read()
            response = make_response(image_data)
            response.headers['Content-Type'] = 'image/png'
            return response

**设置上传文件的接口**

<font color=red size=3>动手练习2</font>


在应用检测图形中，我们需要将图形保存到本地。再将图形进行处理检测，所以需要设置上传文件的保存路径，以及检测完图片后，存放的路径。


- 请在`<1>`中写出原图的访问路径（ip+端口+url路径）`image_url': 'http://127.0.0.1:5003/tmp/ct/' + filename`

- `<2>`中写出检测后图片访问路径（ip+端口+url路径）`draw_url':'http://127.0.0.1:5003/tmp/draw/' + filename`

In [ ]:
@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    file = request.files['file']
    print(datetime.datetime.now(), file.filename)
    if file and allowed_file(file.filename):
        src_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(src_path)
        shutil.copy(src_path, './tmp/ct')
        image_path = os.path.join('./tmp/ct', file.filename)
        filename, image_info = core.main.c_main(
            image_path, current_app.model, file.filename.rsplit('.', 1)[1])
        return jsonify({'status': 1,
                        <1>,
                        <2>,
                        'image_info': image_info})

    return jsonify({'status': 0})

<details>
<summary><font color=red size=3>点击查看动手练习2答案</font></summary>
<pre><code>

```python
@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    file = request.files['file']
    print(datetime.datetime.now(), file.filename)
    if file and allowed_file(file.filename):
        src_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(src_path)
        shutil.copy(src_path, './tmp/ct')
        image_path = os.path.join('./tmp/ct', file.filename)
        filename, image_info = core.main.c_main(
            image_path, current_app.model, file.filename.rsplit('.', 1)[1])
        return jsonify({'status': 1,
                        'image_url': 'http://127.0.0.1:5003/tmp/ct/' + filename,
                        'draw_url': 'http://127.0.0.1:5003/tmp/draw/' + filename,
                        'image_info': image_info})

    return jsonify({'status': 0})
```
</code></pre>
</details>

**运行main函数，启动flask app**

<font color=red size=3>动手练习3</font>


在flask 启动过程中，需要设置启动的ip。启动的端口号。需要用户自己手动配置。默认是`127.0.0.1` 和`5003`端口


请在`<1>`中写出ip,`<2>`中写出端口号

In [ ]:
if __name__ == '__main__':
    files = [
        'uploads', 'tmp/ct', 'tmp/draw',
        'tmp/image', 'tmp/mask', 'tmp/uploads'
    ]
    for ff in files:
        if not os.path.exists(ff):
            os.makedirs(ff)
    with app.app_context():
        current_app.model = Detector()
    app.run(host='127.0.0.1', port=5003)  ##设置app 启动的ip地址 端口 调试模式

<details>
<summary><font color=red size=3>点击查看动手练习3答案</font></summary>
<pre><code>

```python
if __name__ == '__main__':
    files = [
        'uploads', 'tmp/ct', 'tmp/draw',
        'tmp/image', 'tmp/mask', 'tmp/uploads'
    ]
    for ff in files:
        if not os.path.exists(ff):
            os.makedirs(ff)
    with app.app_context():
        current_app.model = Detector()
    app.run(host='127.0.0.1', port=5003)  ##设置app 启动的ip地址 端口 调试模式
```
</code></pre>
</details>

**打开云桌面在浏览器中输入`127.0.0.1::5003`**

出现如下界面说明启动成功

<img style="float: center;" width = "600" height = "200" src="./tmp/image/后端app启动.png">